In [1]:
import pandas as pd
import pandas_datareader as pdr
from bs4 import BeautifulSoup
import requests
import json
import re
import numpy as np
from nltk.corpus import stopwords
from pandas_datareader import data
import datetime

/Users/lincolnjeong/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# prepare codemap

In [2]:
# 종목 타입에 따라 download url이 다름. 종목코드 뒤에 .KS .KQ등이 입력되어야해서 Download Link 구분 필요
stock_type = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt'
    }

# download url 조합
def get_download_stock(market_type=None):
    market_type = stock_type[market_type]
    download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
    download_link = download_link + '?method=download'
    download_link = download_link + '&marketType=' + market_type
    df = pd.read_html(download_link, header=0)[0]
    return df;
# kospi 종목코드 목록 다운로드
def get_download_kospi():
    df = get_download_stock('kospi')
    df.종목코드 = df.종목코드.map('{:06d}'.format)
    return df
# kosdaq 종목코드 목록 다운로드
def get_download_kosdaq():
    df = get_download_stock('kosdaq')
    df.종목코드 = df.종목코드.map('{:06d}'.format)
    return df
# kospi, kosdaq 종목코드 각각 다운로드
kospi_df = get_download_kospi()
kosdaq_df = get_download_kosdaq()
# data frame merge
code_df = pd.concat([kospi_df, kosdaq_df])
# data frame정리
code_df = code_df[['회사명', '종목코드']]
# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

# read headlines(daum news)

In [3]:
p_dict = ['↑', '급상승', '상승', '올랐다', '오름세', '호재', '안착', '대박', '기회', '잡았나', '출발', '반등', '사자', '급등', '승부', '증가', '최고치', '안정적', '등재', 'buy', '개선']
n_dict = ['↓', '상장폐지', '뚝', '관리종목', '주의보', '낚시', '우려', '둔화', '공포', '사임', '어렵다', '어려운 ', '썰렁', '손실', '적자', '과징금', '하락', '약화', '애로']
stopword = ['이다', '너무', '은', '는', '이', '가', '을', '를', '에', '게', '도', '行', '"', "'", '.', '[', ']', ',']

In [4]:
def get_news(date_list):
    news = pd.DataFrame()
    for date in date_list:
        headline = []
        for page in range(1, 1000):
            url = f'https://news.daum.net/breakingnews/economic?page={page}&regDate={date}'
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')

            if len(soup.findAll("a", href=re.compile("https://v.daum.net/v/"))) is 0:
                break
            else:
                for index, link in enumerate(soup.findAll("a", href=re.compile("https://v.daum.net/v/"))):
                    if (link.text != None):
                        headline.append(link.text)

        headlines = set(list(headline))
        df = pd.DataFrame(headlines, columns=['headline'])
        df['date'] = date
        news = pd.concat([news, df])
    return news

In [5]:
def p_scoring(text):
    score = 0
    for keyword in p_dict:
        if keyword in text:
            score += 1
    return score

def n_scoring(text):
    score = 0
    for keyword in n_dict:
        if keyword in text:
            score += 1
    return score

In [6]:
def sw(text):
    for i in stopword:
        if i in text:
            text = text.replace(i, ' ')
    return text

In [7]:
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    try:
        if len(df.query(f"name=='{name}'")['code']) == 0:
            return False
        else:
            code = df.query(f"name=='{name}'")['code'].to_string(index=False)
            # 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
            code = code.strip()
            return code
    except:
        print(name)

In [8]:
def is_code(text):
    tokenization = text.split(' ')
    codes = []
    for token in tokenization:
        if len(token) != 0:
            if get_code(code_df, token.strip()):
                codes.append(get_code(code_df, token))
    if len(codes) == 0:
        return False
    else: return ", ".join(codes)

In [24]:
def change_rate(stock, date):
    day = date
    
    df = print_stock_price(stock)
    
    try:
        today_index = int(df.query(f'date == "{date}"').index.values)
        yesterday_index = int(df.query(f'date == "{date}"').index.values) + 1
        return round(((df.iloc[today_index].adj_close / df.iloc[yesterday_index].adj_close) - 1) * 100, 2)
    except:
        print(stock, date)
    

In [20]:
def print_stock_price(code, page_num=10):
    result = [[], [], []]

    for n in range(page_num):
        url = 'https://finance.naver.com/item/sise_day.nhn?code='+code+'&page='+str(n+1)
        r = requests.get(url)
        html = r.content
        soup = BeautifulSoup(html, 'html.parser')
        tr = soup.select('table > tr')

        for i in range(1, len(tr)-1):
            if tr[i].select('td')[0].text.strip():
                result[0].append(tr[i].select('td')[0].text.strip())
                result[1].append(tr[i].select('td')[1].text.strip())
                result[2].append(tr[i].select('td')[6].text.strip())

    
    df = (pd.DataFrame(result).T)
    df.columns = ['date', 'adj_close', 'volume']
    df.date = df.date.apply(lambda x: x.replace('.', '').strip())
    df.adj_close = df.adj_close.apply(lambda x: int(x.replace(',', '').strip()))
    df.volume = df.volume.apply(lambda x: int(x.replace(',', '').strip()))
    
    return df


stock_code = '018700'
pages = 10

df = print_stock_price(stock_code, pages)

In [9]:
news = get_news(['20200214', '20200215'])
news['p_score'] = news.headline.apply(p_scoring)
news['n_score'] = news.headline.apply(n_scoring)

In [11]:
news['text'] = news.headline.apply(sw)

In [12]:
news['is_code'] = news.text.apply(is_code)

In [39]:
code_news = news.query('is_code != False').reset_index(drop=True)

In [43]:
code_news.shape

(986, 7)

In [50]:
def codes_changes(code_news):
    for idx, row in code_news.iterrows():
        code_news.loc[idx, 'changes'] = change_rate(row.is_code, row.date)
    return code_news

In [ ]:
import parmap 
import multiprocessing
from multiprocessing import Manager 
print(f'cpu cores: {multiprocessing.cpu_count()}')
num_cores = 4 # 사용할 cpu 코어 수. multiprocessing.cpu_count() 로 확인 가능 
manager = Manager() 
#d = manager.dict() 
news_split = np.array_split(code_news, num_cores)

result = parmap.map(codes_changes, news_split, pm_pbar=True, pm_processes=num_cores)


cpu cores: 12


  0%|          | 0/4 [00:00<?, ?it/s]

011510 20200214
012830
 079430 20200214
052640 20200214
084730, 227100 20200214
034730
 003600 20200214
053000
 316140 20200214
021960, 001680 20200214
013730
 234100 20200214
034730
 003600 20200214
052640 20200214
206560, 001040 20200214
052640 20200214
084730, 227100 20200214
005930, 005930 20200214


In [114]:
change_rate('018700', '20200213')

29.88